# Invoice Entity Resolution & Procurement Intelligence

## 🎯 **Business Problem**

**Procurement Challenge:** Every day, purchasing receives 20-50 new vendor quotes that need to be compared against 10,000+ historical invoices to find better pricing and negotiation opportunities.

**Manual Process Problems:**
- ❌ Takes 2-3 hours per quote to research historical purchases
- ❌ Impossible to remember all previous purchases across vendors
- ❌ Miss opportunities to negotiate based on historical pricing
- ❌ No systematic way to identify price increases

**AI Solution:**
- ✅ Semantic search finds similar products across different vendor descriptions
- ✅ Automated price comparison with historical data
- ✅ AI-powered negotiation recommendations
- ✅ Batch processing of daily quotes in minutes
- ✅ Recent purchases weighted higher for relevant pricing

## 🧠 **Technologies Used**
- **Cortex Search**: Semantic similarity matching with recency weighting
- **CORTEX.COMPLETE**: AI-powered price analysis and recommendations
- **Vector Embeddings**: Find similar products across vendor variations

## 📋 **Demo Flow**
1. **Load Data**: Historical invoices + new vendor quotes
2. **Create Search Service**: With recency weighting for better pricing relevance
3. **Single Invoice Analysis**: Deep dive into one quote
4. **Daily Batch Analysis**: Process all quotes to prioritize opportunities


In [ ]:
# Import required packages for data processing
import pandas as pd
import json
from datetime import datetime, timedelta

# Note: In Snowflake notebooks, session and Snowpark functions are automatically available
from snowflake.snowpark import Session

session = Session.builder.getOrCreate()


In [ ]:
# Set up database and schema
session.sql("CREATE DATABASE IF NOT EXISTS DEMODB").collect()
session.sql("USE DATABASE DEMODB").collect()
session.sql("CREATE OR REPLACE SCHEMA procurement_intelligence").collect()
session.sql("USE SCHEMA procurement_intelligence").collect()

print("📍 Database Setup Complete!")
print(f"Database: {session.get_current_database()}")
print(f"Schema: {session.get_current_schema()}")

## 📊 **Step 1: Load Data**

We'll load:
- **Historical Invoices**: 40 previous purchases across different vendors and products
- **New Vendor Quotes**: 10 new quotes that need price analysis

In [ ]:
# Load historical invoices
import pandas as pd

historical_invoices = pd.read_csv('historical_invoices.csv')
print(f"📊 Loaded {len(historical_invoices)} historical invoices")
print(f"💰 Total historical spend: ${historical_invoices['total_amount'].sum():,.2f}")

# Convert to Snowflake table
historical_invoices.columns = [c.upper() for c in historical_invoices.columns]
historical_df = session.create_dataframe(historical_invoices)
historical_df.write.mode("overwrite").save_as_table("HISTORICAL_INVOICES")

print("✅ Historical invoices loaded!")

In [ ]:
# Load new vendor quotes
vendor_quotes = pd.read_csv('vendor_quotes_new.csv')
print(f"📨 Loaded {len(vendor_quotes)} new vendor quotes")
print(f"💸 Total quoted amount: ${vendor_quotes['quoted_total'].sum():,.2f}")

# Convert to Snowflake table
vendor_quotes.columns = [c.upper() for c in vendor_quotes.columns]
quotes_df = session.create_dataframe(vendor_quotes)
quotes_df.write.mode("overwrite").save_as_table("NEW_VENDOR_QUOTES")

print("✅ New quotes loaded!")


## 🔍 **Step 2: Create Search Service with Recency Weighting**

This search service will:
- **Find similar products** across different vendor descriptions
- **Weight recent purchases higher** for more relevant pricing
- **Return key attributes** needed for price comparison

In [ ]:
-- Create search service with recency weighting
-- Note: Using fixed date '2024-12-01' instead of CURRENT_DATE() to avoid change tracking issues
CREATE OR REPLACE CORTEX SEARCH SERVICE PROCUREMENT_SEARCH
  ON SEARCH_TEXT
  ATTRIBUTES INVOICE_ID, VENDOR_NAME, UNIT_PRICE, INVOICE_DATE, RECENCY_WEIGHT
  WAREHOUSE = COMPUTE_WH
  TARGET_LAG = '5 minutes'
  EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
  AS (
    SELECT
        INVOICE_ID,
        
        -- Enhanced search text with recency indicators
        CONCAT(
            'Product: ', PRODUCT_DESCRIPTION,
            ', Category: ', PRODUCT_CATEGORY,
            ', Vendor: ', VENDOR_NAME,
            ', Specs: ', COALESCE(SPECIFICATIONS, ''),
            ', Part: ', COALESCE(PART_NUMBER, ''),
            -- Add recency context for semantic matching
            CASE 
                WHEN INVOICE_DATE >= '2024-06-01' 
                THEN ' [RECENT PURCHASE - CURRENT PRICING]'
                WHEN INVOICE_DATE >= '2023-12-01' 
                THEN ' [RECENT PURCHASE]'
                ELSE ' [OLDER PURCHASE]'
            END
        ) AS SEARCH_TEXT,
        
        VENDOR_NAME,
        UNIT_PRICE,
        INVOICE_DATE,
        
        -- Recency weight: higher for recent purchases
        CASE 
            WHEN INVOICE_DATE >= '2024-09-01' THEN 1.0  -- Last 3 months
            WHEN INVOICE_DATE >= '2024-06-01' THEN 0.9  -- 3-6 months  
            WHEN INVOICE_DATE >= '2023-12-01' THEN 0.7  -- 6-12 months
            WHEN INVOICE_DATE >= '2022-12-01' THEN 0.5  -- 1-2 years
            ELSE 0.3                                    -- 2+ years
        END AS RECENCY_WEIGHT
        
    FROM HISTORICAL_INVOICES
    WHERE INVOICE_DATE >= '2021-12-01'  -- Only last 3 years
  );

SHOW CORTEX SEARCH SERVICES LIKE 'PROCUREMENT_SEARCH';


## 🎯 **Step 3: Analyze Single Invoice Materials**

Let's analyze one specific quote to see how the search finds similar historical purchases and provides negotiation insights.


In [ ]:
-- Analyze materials in one specific quote (QTE001)
-- Using CORTEX_SEARCH_BATCH (the only available search function)

SELECT 
    '🔍 SINGLE QUOTE ANALYSIS: ' || nq.QUOTE_ID as analysis_type,
    nq.PRODUCT_DESCRIPTION as quoted_product,
    nq.VENDOR_NAME as quote_vendor,
    nq.QUOTED_UNIT_PRICE as current_quote,
    
    -- Best historical match from search results
    s.VENDOR_NAME as historical_vendor,
    TRY_CAST(s.UNIT_PRICE AS FLOAT) as historical_price,
    s.INVOICE_DATE as when_purchased,
    TRY_CAST(s.RECENCY_WEIGHT AS FLOAT) as recency_score,
    
    -- Price comparison
    ROUND(((nq.QUOTED_UNIT_PRICE - TRY_CAST(s.UNIT_PRICE AS FLOAT)) / TRY_CAST(s.UNIT_PRICE AS FLOAT) * 100), 1) as price_change_pct,
    (nq.QUOTED_UNIT_PRICE - TRY_CAST(s.UNIT_PRICE AS FLOAT)) * nq.QUOTED_QUANTITY as total_impact,
    
    -- AI recommendation
    SNOWFLAKE.CORTEX.COMPLETE(
        'llama3-70b',
        CONCAT(
            'PROCUREMENT ANALYSIS:\n',
            'Current Quote: $', nq.QUOTED_UNIT_PRICE, ' from ', nq.VENDOR_NAME, '\n',
            'Historical: $', s.UNIT_PRICE, ' from ', s.VENDOR_NAME, ' (', s.INVOICE_DATE, ')\n',
            'Price Change: ', ROUND(((nq.QUOTED_UNIT_PRICE - TRY_CAST(s.UNIT_PRICE AS FLOAT)) / TRY_CAST(s.UNIT_PRICE AS FLOAT) * 100), 1), '%\n',
            'Product: ', nq.PRODUCT_DESCRIPTION, '\n\n',
            'Provide JSON: {"action": "ACCEPT|NEGOTIATE|REJECT", "target_price": 999, "reasoning": "brief explanation", "talking_points": ["point1", "point2"]}'
        )
    ) as ai_recommendation

FROM NEW_VENDOR_QUOTES nq,
LATERAL CORTEX_SEARCH_BATCH(
    service_name => 'PROCUREMENT_SEARCH',
    query => CONCAT('Product: ', nq.PRODUCT_DESCRIPTION, ', Category: ', nq.PRODUCT_CATEGORY, ', Specs: ', COALESCE(nq.SPECIFICATIONS, '')),
    limit => 1
) s
WHERE nq.QUOTE_ID = 'QTE001'  -- Analyze specific quote
  AND s.RANK = 1
  AND s.INVOICE_ID IS NOT NULL
  AND s.ERROR IS NULL;


In [ ]:
-- 📊 ENHANCED SINGLE QUOTE DISPLAY: Top 3 Historical Matches with Clear Recommendations

WITH quote_analysis AS (
    SELECT 
        nq.QUOTE_ID,
        nq.PRODUCT_DESCRIPTION,
        nq.VENDOR_NAME as current_vendor,
        nq.QUOTED_UNIT_PRICE as current_price,
        nq.QUOTED_QUANTITY,
        
        -- Historical match details
        s.RANK as match_rank,
        s.VENDOR_NAME as historical_vendor,
        TRY_CAST(s.UNIT_PRICE AS FLOAT) as historical_price,
        s.INVOICE_DATE as purchase_date,
        TRY_CAST(s.RECENCY_WEIGHT AS FLOAT) as recency_weight,
        
        -- Price analysis
        ROUND(((nq.QUOTED_UNIT_PRICE - TRY_CAST(s.UNIT_PRICE AS FLOAT)) / TRY_CAST(s.UNIT_PRICE AS FLOAT) * 100), 1) as price_variance_pct,
        (nq.QUOTED_UNIT_PRICE - TRY_CAST(s.UNIT_PRICE AS FLOAT)) * nq.QUOTED_QUANTITY as total_price_impact,
        
        -- Recommendation logic
        CASE 
            WHEN ((nq.QUOTED_UNIT_PRICE - TRY_CAST(s.UNIT_PRICE AS FLOAT)) / TRY_CAST(s.UNIT_PRICE AS FLOAT) * 100) > 25 THEN '🔴 NEGOTIATE AGGRESSIVELY'
            WHEN ((nq.QUOTED_UNIT_PRICE - TRY_CAST(s.UNIT_PRICE AS FLOAT)) / TRY_CAST(s.UNIT_PRICE AS FLOAT) * 100) > 15 THEN '🟡 NEGOTIATE'
            WHEN ((nq.QUOTED_UNIT_PRICE - TRY_CAST(s.UNIT_PRICE AS FLOAT)) / TRY_CAST(s.UNIT_PRICE AS FLOAT) * 100) > 5 THEN '🟢 CONSIDER NEGOTIATION'
            WHEN ((nq.QUOTED_UNIT_PRICE - TRY_CAST(s.UNIT_PRICE AS FLOAT)) / TRY_CAST(s.UNIT_PRICE AS FLOAT) * 100) > -5 THEN '✅ FAIR PRICE'
            ELSE '💰 GOOD DEAL'
        END as recommendation
        
    FROM NEW_VENDOR_QUOTES nq,
    LATERAL CORTEX_SEARCH_BATCH(
        service_name => 'PROCUREMENT_SEARCH',
        query => CONCAT('Product: ', nq.PRODUCT_DESCRIPTION, ', Category: ', nq.PRODUCT_CATEGORY, ', Specs: ', COALESCE(nq.SPECIFICATIONS, '')),
        limit => 3  -- Get top 3 matches
    ) s
    WHERE nq.QUOTE_ID = 'QTE001'
      AND s.INVOICE_ID IS NOT NULL
      AND s.ERROR IS NULL
      AND TRY_CAST(s.UNIT_PRICE AS FLOAT) IS NOT NULL
    ORDER BY s.RANK
)

-- Clean display format
SELECT 
    CASE 
        WHEN match_rank = 1 THEN '🎯 QUOTE ANALYSIS: ' || QUOTE_ID || ' - ' || PRODUCT_DESCRIPTION
        ELSE ''
    END as analysis_header,
    
    CASE 
        WHEN match_rank = 1 THEN '💰 CURRENT QUOTE'
        ELSE 'Historical Match #' || match_rank
    END as comparison_type,
    
    CASE 
        WHEN match_rank = 1 THEN current_vendor
        ELSE historical_vendor || ' (' || purchase_date || ')'
    END as vendor_info,
    
    CASE 
        WHEN match_rank = 1 THEN '$' || current_price
        ELSE '$' || historical_price
    END as unit_price,
    
    CASE 
        WHEN match_rank = 1 THEN '$' || (current_price * QUOTED_QUANTITY)
        ELSE '$' || (historical_price * QUOTED_QUANTITY) || ' (if same qty)'
    END as total_cost,
    
    CASE 
        WHEN match_rank = 1 THEN 'BASELINE'
        WHEN price_variance_pct > 0 THEN '+' || price_variance_pct || '% HIGHER'
        ELSE price_variance_pct || '% LOWER'
    END as price_comparison,
    
    CASE 
        WHEN match_rank = 1 THEN recommendation
        ELSE 'Recency Score: ' || ROUND(recency_weight * 100, 0) || '%'
    END as recommendation_or_score

FROM quote_analysis

UNION ALL

-- Summary recommendation
SELECT 
    '🏆 OVERALL RECOMMENDATION' as analysis_header,
    (SELECT recommendation FROM quote_analysis WHERE match_rank = 1) as comparison_type,
    'Based on ' || COUNT(*) || ' historical matches' as vendor_info,
    'Avg Historical: $' || ROUND(AVG(historical_price), 2) as unit_price,
    'Potential Savings: $' || ROUND(ABS(MIN(total_price_impact)), 0) as total_cost,
    CASE 
        WHEN AVG(price_variance_pct) > 15 THEN 'Strong negotiation position'
        WHEN AVG(price_variance_pct) > 5 THEN 'Moderate negotiation opportunity'
        ELSE 'Price appears competitive'
    END as price_comparison,
    'Target Price: $' || ROUND(AVG(historical_price) * 1.05, 2) as recommendation_or_score
FROM quote_analysis;


## 📈 **Step 4: Daily Batch Analysis - All Materials**

Now let's process ALL new quotes for the day to identify the biggest opportunities and priorities.


In [ ]:
-- Step 1: Create batch search results table (following working demo pattern)
CREATE OR REPLACE TABLE PROCUREMENT_BATCH_SEARCH_RESULTS AS
SELECT
  nq.QUOTE_ID,
  nq.PRODUCT_DESCRIPTION,
  nq.VENDOR_NAME as quote_vendor,
  nq.QUOTED_UNIT_PRICE,
  nq.QUOTED_QUANTITY,
  nq.REQUESTED_BY,
  nq.VALID_UNTIL,
  CONCAT('Product: ', nq.PRODUCT_DESCRIPTION, ', Category: ', nq.PRODUCT_CATEGORY, ', Specs: ', COALESCE(nq.SPECIFICATIONS, '')) as search_query,
  s.*
FROM NEW_VENDOR_QUOTES AS nq,
LATERAL CORTEX_SEARCH_BATCH(
  service_name => 'PROCUREMENT_SEARCH',
  query => CONCAT('Product: ', nq.PRODUCT_DESCRIPTION, ', Category: ', nq.PRODUCT_CATEGORY, ', Specs: ', COALESCE(nq.SPECIFICATIONS, '')),
  limit => 3
) AS s;

In [ ]:
-- Step 2: View the batch search results
SELECT * FROM PROCUREMENT_BATCH_SEARCH_RESULTS 
WHERE ERROR IS NULL 
  AND INVOICE_ID IS NOT NULL
ORDER BY QUOTE_ID, RANK
LIMIT 10;


In [ ]:
-- Step 3: Create procurement analysis with clear recommendations
WITH daily_analysis AS (
    SELECT 
        QUOTE_ID,
        PRODUCT_DESCRIPTION,
        quote_vendor,
        QUOTED_UNIT_PRICE,
        QUOTED_QUANTITY,
        REQUESTED_BY,
        VALID_UNTIL,
        
        -- Best historical match (rank 1 only)
        VENDOR_NAME as historical_vendor,
        TRY_CAST(UNIT_PRICE AS FLOAT) as historical_price,
        INVOICE_DATE as historical_date,
        TRY_CAST(RECENCY_WEIGHT AS FLOAT) as recency_weight,
        
        -- Price analysis
        ROUND(((QUOTED_UNIT_PRICE - TRY_CAST(UNIT_PRICE AS FLOAT)) / TRY_CAST(UNIT_PRICE AS FLOAT) * 100), 1) as price_variance_pct,
        (QUOTED_UNIT_PRICE - TRY_CAST(UNIT_PRICE AS FLOAT)) * QUOTED_QUANTITY as potential_savings,
        
        -- Priority classification
        CASE 
            WHEN ((QUOTED_UNIT_PRICE - TRY_CAST(UNIT_PRICE AS FLOAT)) / TRY_CAST(UNIT_PRICE AS FLOAT) * 100) > 20 THEN '🔴 HIGH PRIORITY'
            WHEN ((QUOTED_UNIT_PRICE - TRY_CAST(UNIT_PRICE AS FLOAT)) / TRY_CAST(UNIT_PRICE AS FLOAT) * 100) > 10 THEN '🟡 MEDIUM PRIORITY'
            WHEN ((QUOTED_UNIT_PRICE - TRY_CAST(UNIT_PRICE AS FLOAT)) / TRY_CAST(UNIT_PRICE AS FLOAT) * 100) > 0 THEN '🟢 LOW PRIORITY'
            ELSE '✅ GOOD PRICE'
        END as priority_level
        
    FROM PROCUREMENT_BATCH_SEARCH_RESULTS
    WHERE RANK = 1  -- Only best match
      AND ERROR IS NULL
      AND INVOICE_ID IS NOT NULL
      AND TRY_CAST(UNIT_PRICE AS FLOAT) IS NOT NULL
)

-- Executive Summary
SELECT 
    '📊 DAILY PROCUREMENT INTELLIGENCE SUMMARY' as report_section,
    CAST(COUNT(*) AS VARCHAR) as total_quotes_analyzed,
    CAST(COUNT(CASE WHEN priority_level LIKE '%HIGH%' THEN 1 END) AS VARCHAR) as high_priority_quotes,
    CAST(COUNT(CASE WHEN priority_level LIKE '%MEDIUM%' THEN 1 END) AS VARCHAR) as medium_priority_quotes,
    '$' || CAST(ROUND(SUM(CASE WHEN potential_savings > 0 THEN potential_savings ELSE 0 END), 0) AS VARCHAR) as total_potential_savings,
    CAST(ROUND(AVG(price_variance_pct), 1) AS VARCHAR) || '%' as avg_price_variance_pct
FROM daily_analysis

UNION ALL

-- Top Priority Items (biggest opportunities)
SELECT 
    priority_level as report_section,
    QUOTE_ID as total_quotes_analyzed,
    PRODUCT_DESCRIPTION as high_priority_quotes,
    '$' || CAST(ROUND(ABS(potential_savings), 0) AS VARCHAR) || ' savings opportunity' as medium_priority_quotes,
    CAST(price_variance_pct AS VARCHAR) || '% vs historical ($' || CAST(historical_price AS VARCHAR) || ')' as total_potential_savings,
    'Expires: ' || CAST(VALID_UNTIL AS VARCHAR) as avg_price_variance_pct
FROM daily_analysis
WHERE priority_level IN ('🔴 HIGH PRIORITY', '🟡 MEDIUM PRIORITY')
ORDER BY 
    CASE WHEN report_section LIKE '%SUMMARY%' THEN 1 
         WHEN report_section LIKE '%HIGH%' THEN 2 
         ELSE 3 END
LIMIT 15;
